In [1]:
import os

import sys
sys.path.insert(0, '..')

from sequencing_process.process_fastq_gz import align_fastq_gzs_using_hisat2
from sequencing_process.process_bam import call_variants_on_bam_using_freebayes_and_multiprocess
from sequencing_process.process_vcf_gz import annotate_vcf_gz_using_snpeff, annotate_vcf_gz_using_bcftools, filter_vcf_gz_using_bcftools

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
GRCH_DIRECTORY_PATH = '/media/k/Chocolate/Jumis/data/grch'
assert os.path.isdir(GRCH_DIRECTORY_PATH)

FASTA_FILE_PATH = os.path.join(GRCH_DIRECTORY_PATH,
                               'Homo_sapiens.GRCh38.dna.primary_assembly.fa')
assert os.path.isfile(FASTA_FILE_PATH)

CLINVAR_VCF_GZ_FILE_PATH = os.path.join(GRCH_DIRECTORY_PATH, 'clinvar.vcf.gz')
assert os.path.isfile(CLINVAR_VCF_GZ_FILE_PATH)

In [10]:
FASTQ_GZ_0_FILE_PATH = None
FASTQ_GZ_1_FILE_PATH = None

N_JOBS = 6

In [15]:
BAM_FILE_PATH = None

if not BAM_FILE_PATH:

    BAM_FILE_PATH = align_fastq_gz_with_hisat2(
        FASTA_FILE_PATH, [
            FASTQ_GZ_0_FILE_PATH,
            FASTQ_GZ_1_FILE_PATH,
        ],
        'DNA',
        n_jobs=N_JOBS)
    

In [6]:
VCF_GZ_FILE_PATH = '/media/k/Silver/people/738/genome.filter_vcf_gz_using_bcftools.annotate_vcf_gz_using_snpeff.vcf.gz'

if not VCF_GZ_FILE_PATH:

    VCF_GZ_FILE_PATH = call_variants_on_bam_using_freebayes_and_multiprocess(
        BAM_FILE_PATH, FASTA_FILE_PATH, n_jobs=N_JOBS)

    n = os.path.join(os.path.dirname(VCF_GZ_FILE_PATH), 'genome.vcf.gz')
    os.rename(VCF_GZ_FILE_PATH, n)
    os.rename(VCF_GZ_FILE_PATH + '.tbi', n + '.tbi')
    VCF_GZ_FILE_PATH = n

    VCF_GZ_FILE_PATH = annotate_vcf_gz_using_snpeff(
        VCF_GZ_FILE_PATH, n_jobs=N_JOBS)
    
    VCF_GZ_FILE_PATH = annotate_vcf_gz_using_bcftools(
        VCF_GZ_FILE_PATH,
        CLINVAR_VCF_GZ_FILE_PATH,
        '--columns =ID,INFO',
        n_jobs=N_JOBS)
    
    VCF_GZ_FILE_PATH = filter_vcf_gz_using_bcftools(
        VCF_GZ_FILE_PATH, n_jobs=N_JOBS)